# Section 2: The building blocks of a Scientific Computing Platform


## Goal 1 - Scalable, interactice compute

## Goal 2 - Reproducible Research

## Goal 3 - Shareable research

## Goal 4 - Cost effective compute

## Goal 5 - Separation of concerns
Allowing researchers to focus on the goals of their research project and spend minimal time worrying about the underlying enabling infrastructure.